# Joint State Demo

>This is an example of how we can quickly build a standalone web app for our robot using ipywidgets and Voilà.

Make sure there is a robot published, you can launch panda demo with the following command:

`roslaunch {Notebook Example Folder}/launch/panda_demo.launch`

In [2]:
!rosnode kill joint_state_publisher

In [ ]:
import random

import ipywidgets as wid
import rospy as rp
from sensor_msgs.msg import JointState

from ipywidgets import FloatSlider, Layout, Button, Box, FloatText, Textarea, Dropdown, Label, IntSlider

In [ ]:
from joint_state_publisher import JointStatePublisher

In [ ]:
jsp = JointStatePublisher()

In [ ]:
rp.init_node('supernode')
msg = JointState()

In [ ]:
from joint_state_publisher import JointStatePublisher
msg_joint = JointStatePublisher()

In [ ]:
#name = ["shoulder_pan_joint","shoulder_lift_joint","elbow_joint",'wirst_1_joint','wirst_2_joint','wirst_3_joint']

joint_state_dict = {}
for i in jsp.free_joints:
    minPos = jsp.free_joints[i]['min']
    maxPos = jsp.free_joints[i]['max']
    step = (maxPos - minPos) / 100
    joint_state_dict[i] = FloatSlider(min = minPos, max = maxPos, step = step)
    
# joint_state_dict

In [ ]:
pub = rp.Publisher('/joint_states', JointState, queue_size=10)
def on_sent(b):
    msg.header.stamp = rp.Time.now()
    msg.name = [str(i) for i in joint_state_dict]
    msg.position = [joint_state_dict[i].value for i in joint_state_dict]
    pub.publish(msg)

for key, value in joint_state_dict.items():
    value.observe(on_sent)

In [ ]:
randomize = Button(description='Randomize')

def on_randomize(randomize):
    pub = rp.Publisher('/joint_states', JointState, queue_size=10)
    for i in joint_state_dict:
        msg.name = [str(i)]
        msg.position = [random.uniform(joint_state_dict[i].min, joint_state_dict[i].max)]
        joint_state_dict[i].value = msg.position[0]
        pub.publish(msg)    
        
randomize.on_click(on_randomize)

In [ ]:
center = Button(description='Center')

def on_center(b):
    pub = rp.Publisher('/joint_states', JointState, queue_size=10)
    for i in joint_state_dict:
        msg.name = [str(i)]
        msg.position = [0]
        joint_state_dict[i].value = 0
        pub.publish(msg)

center.on_click(on_center)

In this case, we are using Zethus to simulate the robot but you can connect a real robot to ros and use this form to control it.

This is just a preview but you could launch a server with the standalone app using the command `voila joint_state_demo.ipynb`

In [ ]:
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items1 = [Box([Label(value= key), joint_state_dict[key]]) for key in jsp.free_joints]
form_items1 += [Box([center]), Box([randomize])]

form = Box(form_items1, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    #width='50%',
    padding='10px'
))

form